In [36]:
import pandas as pd
import numpy as np
from data_processor import *
from lda_model import LDATopicModel
from classifier import TopicClassifier
from visualizer import *
from brain_visualizer import *

inp_df = pd.read_csv('C:/Users/BREIN/Desktop/copathology_visualization_temp/data/260120_wsev_smc_combined_zscores.csv')

# inp_df = inp_df[~inp_df["DX"].isin(['svPPA', 'nfvPPA'])] #######################
print(inp_df['DX'].value_counts())


df_nacc_resilience = pd.read_csv('C:/Users/BREIN/Desktop/copathology_visualization_temp/data/nacc/NACC_resilience_inference.csv')
df_adni4_resilience = pd.read_csv('C:/Users/BREIN/Desktop/copathology_visualization_temp/data/adni/ADNI4_resilience_inference.csv')
df_adni4_resilience = df_adni4_resilience.rename(columns={"FULL_ID": "SUBJ_ID"})
df_nacc_resilience = df_nacc_resilience.rename(columns={"subject_id" : "SUBJ_ID"})


DX
AD        72
svPPA     59
bvFTD     53
nfvPPA    46
DLB       25
PD        24
SVAD      24
Name: count, dtype: int64


In [ ]:
# LDA, CLASS-WEIGHTED XGBoost - RETIRED
N_TOPICS = 6 ###
region_cols = list(inp_df.loc[:, "VA/2":"VA/2035"].columns)
labels = inp_df["DX"].values
ids = inp_df["SUBJ_ID"].values

# Fit LDA on combined z-scores
lda = LDATopicModel(n_topics=N_TOPICS)
theta = lda.fit_transform(inp_df[region_cols])

# Fit classifier
classifier = TopicClassifier(n_splits=5, class_weight='balanced') ##
cv_results = classifier.cross_validate(theta, labels, ids, verbose=True)
classifier.fit(theta, labels)

print(f"CV Accuracy: {cv_results['accuracy']:.4f}")

visualizer = CopathologyVisualizer(
    output_dir=f'./model_parameter_tuning_results/topics_{N_TOPICS}_class_weighted'
)

fig_conf_mat = visualizer.plot_confusion_matrix(
    cm=classifier.get_confusion_matrix(),
    class_names=classifier._classes
)

fig_top_regions = visualizer.plot_top_regions_per_topic(
    topic_patterns = lda.get_topic_patterns(),
    region_names=region_cols
)

fig3 = visualizer.plot_diagnosis_topic_profiles(
    theta=lda._theta,
    dx_labels = labels
)

In [37]:
# Class Balanced - Downsample to 25
## DOWNSAMPLE LARGE DX 
N = 25
dx_col = "DX"
balanced_parts = []

for dx, g in inp_df.groupby(dx_col):
    # if dx == 'AD':
    #     N=50
    # else: 
    #     N=25
    if len(g) > N:
        g = g.sample(n=N, replace=False, random_state=42)
    balanced_parts.append(g)

balanced_df = pd.concat(balanced_parts).reset_index(drop=True)
print(balanced_df[dx_col].value_counts())
####

# for n in list(range(6, 25, 2)):
n=20
print('k_topics = ', n)
N_TOPICS = n ###
region_cols = list(balanced_df.loc[:, "VA/2":"VA/2035"].columns)
labels = balanced_df["DX"].values
ids = balanced_df["SUBJ_ID"].values

# Fit LDA on combined z-scores
lda = LDATopicModel(n_topics=N_TOPICS)
theta = lda.fit_transform(balanced_df[region_cols])

# Fit classifier
classifier = TopicClassifier(n_splits=5) ##
cv_results = classifier.cross_validate(theta, labels, ids, verbose=False)
classifier.fit(theta, labels)

print(f"K_topics {n}, CV Accuracy: {cv_results['accuracy']:.4f}")

# visualizer = CopathologyVisualizer(
#     output_dir=f'./model_parameter_tuning_results/topics_{N_TOPICS}_downsampled'
# )

# fig_conf_mat = visualizer.plot_confusion_matrix(
#     cm=classifier.get_confusion_matrix(),
#     class_names=classifier._classes
# )

# fig_top_regions = visualizer.plot_top_regions_per_topic(
#     topic_patterns = lda.get_topic_patterns(),
#     region_names=region_cols
# )

# fig3 = visualizer.plot_diagnosis_topic_profiles(
#     theta=lda._theta,
#     dx_labels = labels
# )

DX
AD        25
DLB       25
bvFTD     25
nfvPPA    25
svPPA     25
PD        24
SVAD      24
Name: count, dtype: int64
k_topics =  20


c:\Users\BREIN\anaconda3\envs\data_management\lib\site-packages\xgboost\training.py:183: UserWarning: [14:14:53] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\BREIN\anaconda3\envs\data_management\lib\site-packages\xgboost\training.py:183: UserWarning: [14:14:54] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\BREIN\anaconda3\envs\data_management\lib\site-packages\xgboost\training.py:183: UserWarning: [14:14:54] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


K_topics 20, CV Accuracy: 0.4509
Object `Copatholo` not found.


In [ ]:
print(lda._theta.shape)

In [ ]:
## Surface Mapping 
## Topicwise Surface Maps
from atlas_vis import DKTAtlas62ROIPlotter
plotter_62  = DKTAtlas62ROIPlotter(
    cmap='Reds',
    clim=(0, 0.1),  
    window_size=(1200, 1000),
    nan_color='lightgray',
    background='white',
    template_key='pial'
)
os.makedirs(f'./model_parameter_tuning_results/topics_{n}_downsampled/topicwise',exist_ok=True)

# df = pd.read_csv('./combined_results//lda_topic_atrophy_patterns.csv')
# df = lda.get_topic_patterns().T
topic_df = pd.DataFrame(
    lda.get_topic_patterns().T,
    index=region_cols,
    columns=[f"Topic_{k}" for k in range(n)]
)

df = topic_df.tail(62).reset_index(drop=True)

print(len(df))
for col in df.columns: ##################
    print(col)
    l_values = df.loc[:30,col].to_list()
    r_values = df.loc[31:,col].to_list()
    print(len(l_values))
    print(len(r_values))
    print(np.min(l_values+r_values))
    print(np.max(l_values+r_values))

    plotter_62(l_values, r_values, save_path=f'./model_parameter_tuning_results/topics_{n}_downsampled/topicwise/{col}.png')


**NACC INFERENCE**

In [ ]:
## NACC Inference ## 260120
from data_processor import *
nacc_raw = pd.read_csv('C:/Users/BREIN/Desktop/copathology_visualization_temp/data/nacc/260120_NACC_VA_TAU_PATH_matched.csv')
# nacc_raw.rename(columns=col_map)
region_cols = nacc_raw.loc[:, 'VA/2':'VA/2035'].columns
pathology_cols = nacc_raw.loc[:, 'NACC_AD':'NACC_svPPA'].columns
nacc_filtered = nacc_raw[nacc_raw['DX'] != 'Unknown']

nacc_cn = nacc_filtered[nacc_filtered['DX'] == 'CN']
nacc_pat = nacc_filtered[nacc_filtered['DX'] != 'CN']
# nacc_pat = nacc_filtered

print(nacc_cn.shape)
print(nacc_pat.shape)

nacc_prep = DataProcessor(
    region_cols=region_cols,
    dx_col='DX',
    subject_col='subject_id'
)
nacc_prep.fit_baseline(hc_data=nacc_cn)
nacc_Z = nacc_prep.compute_atrophy_scores(data=nacc_pat)
print(type(nacc_Z))

nacc_theta = lda.transform(nacc_Z)
y_pred = classifier.predict(nacc_theta)
y_proba = classifier.predict_proba(nacc_theta)
# print(nacc_theta.shape)
# print(y_pred.shape)
# print(y_proba.shape)

nacc_results = pd.DataFrame(nacc_theta, columns=[f"Topic_{k}" for k in range(lda.n_topics)])
print(nacc_results.shape)

subj_col = nacc_prep.subject_col
if subj_col in nacc_pat.columns:
    nacc_results.insert(0, "SUBJ_ID", nacc_pat[subj_col].values)

nacc_results['pred_DX'] = y_pred
for i, dx in enumerate(classifier.classes):
    nacc_results[f"P({dx})"] = y_proba[:,i]

nacc_results = nacc_results.merge(
    df_nacc_resilience[["SUBJ_ID", "TN_group"]],
    on="SUBJ_ID",
    how="left"
)
nacc_results = nacc_results.dropna(subset=['TN_group'])

nacc_results = nacc_results.merge(
    nacc_raw[["subject_id", "DX"]],
    left_on="SUBJ_ID",
    right_on="subject_id",
    how="left"
)

nacc_results = nacc_results.drop(columns=["subject_id"])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from math import ceil
from itertools import cycle
def plot_dx_radar(df, dx_col="DX", topic_prefix="Topic_", min_n=5, ncols=4):
    topic_cols = [c for c in df.columns if c.startswith(topic_prefix)]
    angles = np.linspace(0, 2*np.pi, len(topic_cols), endpoint=False)
    angles = np.r_[angles, angles[0]]

    # valid DX groups
    groups = [
        (dx, (df[dx_col] == dx).sum())
        for dx in sorted(df[dx_col].dropna().unique())
        if (df[dx_col] == dx).sum() >= min_n
    ]

    if not groups:
        print("No DX groups meet minimum subject requirement.")
        return

    def profile(d):
        v = d[topic_cols].mean().values
        return np.r_[v, v[0]]

    colors = plt.cm.tab10.colors

    # ===============================
    # Individual radar panels
    # ===============================
    nrows = ceil(len(groups) / ncols)
    fig, axes = plt.subplots(
        nrows, ncols,
        figsize=(4*ncols, 4*nrows),
        subplot_kw=dict(polar=True)
    )

    axes = np.array(axes).reshape(-1)
    color_cycle = cycle(colors)

    for ax, (dx, n) in zip(axes, groups):
        vals = profile(df[df[dx_col] == dx])
        c = next(color_cycle)

        ax.plot(angles, vals, lw=2.5, color=c)
        ax.fill(angles, vals, alpha=0.25, color=c)
        ax.set_thetagrids(angles[:-1]*180/np.pi, topic_cols)
        ax.set_title(f"{dx}\n(n={n})", fontsize=11)

    for ax in axes[len(groups):]:
        ax.remove()

    plt.suptitle("Topic Profiles by Diagnosis", y=1.02, fontsize=15)
    plt.tight_layout()
    plt.show()

    # # ===============================
    # # Combined overlay
    # # ===============================
    # plt.figure(figsize=(8, 8))
    # ax = plt.subplot(111, polar=True)

    # for dx, n in groups:
    #     vals = profile(df[df[dx_col] == dx])
    #     c = next(cycle(colors))
    #     ax.plot(angles, vals, lw=2.5, color=c)
    #     ax.fill(angles, vals, alpha=0.12, color=c)

    # ax.set_thetagrids(angles[:-1]*180/np.pi, topic_cols)
    # ax.set_title("Combined Topic Profiles", pad=25)
    # plt.tight_layout()
    # plt.show()

plot_dx_radar(nacc_results)


In [ ]:
## Pathology-wise Radar Plot
import numpy as np
import matplotlib.pyplot as plt
from math import ceil
from itertools import cycle

viz_df = nacc_results.merge(
    nacc_pat[[nacc_prep.subject_col] + list(pathology_cols)],
    left_on="SUBJ_ID",
    right_on=nacc_prep.subject_col,
    how="left"
)
# -----------------------------
# Topic label map
# -----------------------------

topic_cols = [c for c in viz_df.columns if c.startswith('Topic_')]

# -----------------------------
# Radar utilities
# -----------------------------
def compute_topic_profile(df):
    values = df[topic_cols].mean().values
    return np.concatenate([values, [values[0]]])


def radar_angles(n):
    angles = np.linspace(0, 2 * np.pi, n, endpoint=False)
    return np.concatenate([angles, [angles[0]]])


# -----------------------------
# Main visualization
# -----------------------------
def plot_pathology_radar_panels(
    viz_df,
    pathology_cols,
    min_subjects=1,
    ncols=4
):
    """
    Creates:
    1) Grid of individual pathology radar plots
    2) One large combined radar plot
    """

    # -------------------------
    # collect valid pathologies
    # -------------------------
    valid = []
    for p in pathology_cols:
        n = (viz_df[p] == 1).sum()
        if n >= min_subjects:
            valid.append((p, n))

    if len(valid) == 0:
        print("No pathology columns with positive subjects.")
        return

    angles = radar_angles(len(topic_cols))
    colors = plt.cm.tab10.colors
    color_cycle = cycle(colors)

    # ======================================================
    # 1. INDIVIDUAL RADAR SUBPLOTS
    # ======================================================
    n_panels = len(valid)
    nrows = ceil(n_panels / ncols)

    fig, axes = plt.subplots(
        nrows=nrows,
        ncols=ncols,
        figsize=(4 * ncols, 4 * nrows),
        subplot_kw=dict(polar=True)
    )

    axes = np.array(axes).reshape(-1)

    for ax, (pathology, n_pos) in zip(axes, valid):

        pos_df = viz_df[viz_df[pathology] == 1]
        values = compute_topic_profile(pos_df)
        color = next(color_cycle)

        ax.plot(angles, values, linewidth=2.5, color=color)
        ax.fill(angles, values, alpha=0.25, color=color)

        ax.set_thetagrids(
            angles[:-1] * 180 / np.pi,
            topic_cols,
            fontsize=10
        )

        ax.set_title(f"{pathology}\n(n={n_pos})", fontsize=11, pad=12)

    # remove unused axes
    for ax in axes[len(valid):]:
        ax.remove()

    fig.suptitle(
        "Topic Expression by Copathology (Positive Subjects Only)",
        fontsize=16,
        y=1.02
    )

    plt.tight_layout()
    plt.show()

    # ======================================================
    # 2. COMBINED RADAR OVERLAY
    # ======================================================
    plt.figure(figsize=(9, 9))
    ax = plt.subplot(111, polar=True)

    legend_labels = []
    color_cycle = cycle(colors)

    for pathology, n_pos in valid:

        pos_df = viz_df[viz_df[pathology] == 1]
        values = compute_topic_profile(pos_df)
        color = next(color_cycle)

        ax.plot(angles, values, linewidth=2.5, color=color)
        ax.fill(angles, values, alpha=0.12, color=color)

        legend_labels.append(f"{pathology} (n={n_pos})")

    ax.set_thetagrids(
        angles[:-1] * 180 / np.pi,
        topic_cols,
        fontsize=12
    )

    ax.set_title(
        "Combined Topic Profiles Across Copathologies",
        fontsize=16,
        pad=30
    )

    ax.legend(
        legend_labels,
        bbox_to_anchor=(1.35, 1.1),
        loc="upper right",
        frameon=False
    )

    plt.tight_layout()
    plt.show()
plot_pathology_radar_panels(
    viz_df=viz_df,
    pathology_cols=pathology_cols,
    ncols=4
)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# prob_cols = nacc_results.loc[:,'P(AD)':'P(svPPA)'].columns
# nacc_results = nacc_results[nacc_results['DX']=='AD']######## TEMP
prob_cols = nacc_results.loc[:,'P(AD)':'P(SVAD)'].columns
group_col = 'TN_group'
# ------------------------------------------------------------
# Compute group-wise mean probabilities
# ------------------------------------------------------------
group_means = (
    nacc_results
    .groupby('TN_group')[prob_cols]
    .mean()
)

# ------------------------------------------------------------
# Plot heatmap
# ------------------------------------------------------------
plt.figure(figsize=(8, 5))

sns.heatmap(
    group_means,
    cmap="Reds",
    annot=True,
    fmt=".2f",
    linewidths=0.5,
    vmin=0,
    vmax=0.4,
    cbar_kws={"label": "Mean predicted probability"}
)

plt.xlabel("Predicted pathology")
plt.ylabel("Subgroup")
plt.title("Group-wise Mean Predicted Probability Distribution")

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# prob_cols = nacc_results.loc[:, 'P(AD)':'P(svPPA)'].columns
prob_cols = nacc_results.loc[:, 'P(AD)':'P(SVAD)'].columns
group_col = "TN_group"

# ------------------------------------------------------------
# Sort: group first, then descending P(AD)
# ------------------------------------------------------------
nacc_sorted = (
    nacc_results
    .sort_values([group_col, "P(AD)"], ascending=[True, False])
    .reset_index(drop=True)
)

heatmap_data = nacc_sorted[prob_cols]

# ------------------------------------------------------------
# Compute group positions for y-axis labels
# ------------------------------------------------------------
group_counts = nacc_sorted[group_col].value_counts(sort=False)

group_centers = {}
start = 0

for grp, count in group_counts.items():
    center = start + count / 2
    group_centers[grp] = center
    start += count

# ------------------------------------------------------------
# Plot
# ------------------------------------------------------------
plt.figure(figsize=(10, 10))

ax = sns.heatmap(
    heatmap_data,
    cmap="Reds",
    vmin=0,
    vmax=1,
    yticklabels=False,
    cbar_kws={"label": "Predicted probability"}
)

# ------------------------------------------------------------
# Horizontal lines between groups
# ------------------------------------------------------------
cum_sizes = np.cumsum(group_counts.values)

for y in cum_sizes[:-1]:
    ax.hlines(y, *ax.get_xlim(), colors="black", linewidth=1.5)

# ------------------------------------------------------------
# TN subgroup labels on y-axis
# ------------------------------------------------------------
ax.set_yticks(list(group_centers.values()))
ax.set_yticklabels(list(group_centers.keys()), rotation=0, fontsize=11)

# ------------------------------------------------------------
# Labels
# ------------------------------------------------------------
ax.set_xlabel("Predicted pathology")
ax.set_ylabel("TN subgroup")
ax.set_title("Subject-level Predicted Probability Heatmap\n(sorted by descending P(AD))")

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

topic_cols = [c for c in nacc_results.columns if c.startswith("Topic_")]
group_col = "TN_group"

groups = nacc_results[group_col].unique()
n_groups = len(groups)
n_topics = len(topic_cols)

# ------------------------------------------------------------
# Global max for shared axis
# ------------------------------------------------------------
global_max = (
    nacc_results
    .groupby(group_col)[topic_cols]
    .mean()
    .values
    .max()
)

# ------------------------------------------------------------
# Radar setup
# ------------------------------------------------------------
angles = np.linspace(0, 2 * np.pi, n_topics, endpoint=False)
angles = np.concatenate([angles, [angles[0]]])

fig, axes = plt.subplots(
    1, n_groups,
    figsize=(4 * n_groups, 4),
    subplot_kw=dict(polar=True)
)

if n_groups == 1:
    axes = [axes]

# ------------------------------------------------------------
# Plot
# ------------------------------------------------------------
for ax, grp in zip(axes, groups):

    grp_df = nacc_results[nacc_results[group_col] == grp]
    mean_topics = grp_df[topic_cols].mean().values
    mean_topics = np.concatenate([mean_topics, [mean_topics[0]]])

    ax.plot(angles, mean_topics, linewidth=2)
    ax.fill(angles, mean_topics, alpha=0.25)

    ax.set_title(grp, pad=20)

    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(topic_cols, fontsize=9)

    ax.set_ylim(0, global_max * 1.1)   # ✅ shared scale
    ax.set_yticklabels([])

plt.suptitle("TN Subgroup Topic Weight Profiles (shared radial scale)", fontsize=14)
plt.tight_layout()
plt.show()


In [ ]:
## NACC vulnerable check
print('VULN')
r2 = nacc_results[nacc_results['TN_group']=='Vulnerable']
print(r2['DX'].value_counts())
print('\nRESIL')
r2 = nacc_results[nacc_results['TN_group']=='Resilient']
print(r2['DX'].value_counts())
print('\nCANON')
r2 = nacc_results[nacc_results['TN_group']=='Canonical']
print(r2['DX'].value_counts())

**ADNI INFERENCE**

In [ ]:
## ADNI4 Inference ## 260120
adni4_raw = pd.read_csv('C:/Users/BREIN/Desktop/copathology_visualization_temp/data/stage_data/ptau_volume_model/ADNI4_3.csv')
region_cols = adni4_raw.loc[:, 'VA/2':'VA/2035'].columns
adni4_raw = adni4_raw.dropna(subset=region_cols)
adni4_cn = adni4_raw[adni4_raw['DX'] == 'CN']
adni4_pat = adni4_raw[adni4_raw['DX'] != 'CN']
adni4_stage_df = adni4_pat[['FULL_ID', 'DX', 'tau_stage_aa/low', 'tau_stage_aa/mid', 'tau_stage_aa/high', 'pred_tau_stage_aa/low', 'pred_tau_stage_aa/mid', 'pred_tau_stage_aa/high']].dropna()
prob_cols = ['pred_tau_stage_aa/low','pred_tau_stage_aa/mid','pred_tau_stage_aa/high']
max_col = adni4_stage_df[prob_cols].idxmax(axis=1)
adni4_stage_df[prob_cols] = 0
adni4_stage_df.loc[:, prob_cols] = (pd.get_dummies(max_col).reindex(columns=prob_cols, fill_value=0).astype(float))
stage_map = {
    'low': 0,
    'mid': 1,
    'high': 2
}
def get_stage(colname):
    return stage_map[colname.split('/')[-1]]
adni4_stage_df['gt_stage'] = (
    adni4_stage_df[['tau_stage_aa/low', 'tau_stage_aa/mid', 'tau_stage_aa/high']]
    .idxmax(axis=1)
    .apply(get_stage)
)

adni4_stage_df['pred_stage'] = (
    adni4_stage_df[prob_cols]
    .idxmax(axis=1)
    .apply(get_stage)
)

# --------------------------------
# Subject grouping
# --------------------------------
adni4_lower_than_pred = adni4_stage_df[
    adni4_stage_df['gt_stage'] < adni4_stage_df['pred_stage']
][['FULL_ID', 'DX', 'gt_stage', 'pred_stage']]

adni4_exact_match = adni4_stage_df[
    adni4_stage_df['gt_stage'] == adni4_stage_df['pred_stage']
][['FULL_ID', 'DX', 'gt_stage', 'pred_stage']]



adni4_prep = DataProcessor(
    region_cols=region_cols,
    dx_col='DX',
    subject_col='FULL_ID'
)
adni4_prep.fit_baseline(hc_data=adni4_cn)
adni4_Z = adni4_prep.compute_atrophy_scores(data=adni4_pat)
print(adni4_Z.shape)
print(adni4_cn.shape)

adni4_theta = lda.transform(adni4_Z)
adni4_y_pred = classifier.predict(adni4_theta)
adni4_y_proba = classifier.predict_proba(adni4_theta)

adni4_results = pd.DataFrame(adni4_theta, columns=[f"Topic_{k}" for k in range(lda.n_topics)])
print(adni4_results.shape)

subj_col = adni4_prep.subject_col
if subj_col in adni4_pat.columns:
    adni4_results.insert(0, "SUBJ_ID", adni4_pat[subj_col].values)

adni4_results['pred_DX'] = adni4_y_pred
for i, dx in enumerate(classifier.classes):
    adni4_results[f"P({dx})"] = adni4_y_proba[:,i]

adni4_results = adni4_results.merge(
    adni4_raw[["FULL_ID", "DX"]],
    left_on="SUBJ_ID",
    right_on="FULL_ID",
    how="left"
)
adni4_results = adni4_results.drop(columns=["FULL_ID"])

adni4_results["SUBJ_ID"] = adni4_results["SUBJ_ID"].str.replace("_M", "_m", regex=False)

adni4_results = adni4_results.merge(
    df_adni4_resilience[["SUBJ_ID", "TN_group"]],
    on="SUBJ_ID",
    how="left"
)
adni4_results = adni4_results.dropna(subset=['TN_group'])
print('!!!!', adni4_results.shape)

In [ ]:
plot_dx_radar(adni4_results)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

adni4_results = adni4_results[adni4_results['DX']=='AD']######## TEMP

prob_cols = adni4_results.loc[:,'P(AD)':'P(bvFTD)'].columns

group_col = 'TN_group'
# ------------------------------------------------------------
# Compute group-wise mean probabilities
# ------------------------------------------------------------
group_means = (
    adni4_results
    .groupby('TN_group')[prob_cols]
    .mean()
)

# ------------------------------------------------------------
# Plot heatmap
# ------------------------------------------------------------
plt.figure(figsize=(8, 5))

sns.heatmap(
    group_means,
    cmap="Reds",
    annot=True,
    fmt=".2f",
    linewidths=0.5,
    vmin=0,
    vmax=0.4,
    cbar_kws={"label": "Mean predicted probability"}
)

plt.xlabel("Predicted pathology")
plt.ylabel("Subgroup")
plt.title("ADNI4 Group-wise Mean Predicted Probability Distribution")

plt.tight_layout()
plt.show()


In [ ]:
prob_cols = adni4_results.loc[:, 'P(AD)':'P(bvFTD)'].columns
group_col = "TN_group"

# ------------------------------------------------------------
# Sort: group first, then descending P(AD)
# ------------------------------------------------------------
adni4_sorted = (
    adni4_results
    .sort_values([group_col, "P(AD)"], ascending=[True, False])
    .reset_index(drop=True)
)

heatmap_data = adni4_sorted[prob_cols]

# ------------------------------------------------------------
# Compute group positions for y-axis labels
# ------------------------------------------------------------
group_counts = adni4_sorted[group_col].value_counts(sort=False)

group_centers = {}
start = 0

for grp, count in group_counts.items():
    center = start + count / 2
    group_centers[grp] = center
    start += count

# ------------------------------------------------------------
# Plot
# ------------------------------------------------------------
plt.figure(figsize=(10, 10))

ax = sns.heatmap(
    heatmap_data,
    cmap="Reds",
    vmin=0,
    vmax=1,
    yticklabels=False,
    cbar_kws={"label": "Predicted probability"}
)

# ------------------------------------------------------------
# Horizontal lines between groups
# ------------------------------------------------------------
cum_sizes = np.cumsum(group_counts.values)

for y in cum_sizes[:-1]:
    ax.hlines(y, *ax.get_xlim(), colors="black", linewidth=1.5)

# ------------------------------------------------------------
# TN subgroup labels on y-axis
# ------------------------------------------------------------
ax.set_yticks(list(group_centers.values()))
ax.set_yticklabels(list(group_centers.keys()), rotation=0, fontsize=11)

# ------------------------------------------------------------
# Labels
# ------------------------------------------------------------
ax.set_xlabel("Predicted pathology")
ax.set_ylabel("TN subgroup")
ax.set_title("ADNI4 Subject-level Predicted Probability Heatmap\n(sorted by descending P(AD))")

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

topic_cols = [c for c in adni4_results.columns if c.startswith("Topic_")]
group_col = "TN_group"

groups = adni4_results[group_col].unique()
n_groups = len(groups)
n_topics = len(topic_cols)

# ------------------------------------------------------------
# Global max for shared axis
# ------------------------------------------------------------
global_max = (
    adni4_results
    .groupby(group_col)[topic_cols]
    .mean()
    .values
    .max()
)

# ------------------------------------------------------------
# Radar setup
# ------------------------------------------------------------
angles = np.linspace(0, 2 * np.pi, n_topics, endpoint=False)
angles = np.concatenate([angles, [angles[0]]])

fig, axes = plt.subplots(
    1, n_groups,
    figsize=(4 * n_groups, 4),
    subplot_kw=dict(polar=True)
)

if n_groups == 1:
    axes = [axes]

# ------------------------------------------------------------
# Plot
# ------------------------------------------------------------
for ax, grp in zip(axes, groups):

    grp_df = adni4_results[adni4_results[group_col] == grp]
    mean_topics = grp_df[topic_cols].mean().values
    mean_topics = np.concatenate([mean_topics, [mean_topics[0]]])

    ax.plot(angles, mean_topics, linewidth=2)
    ax.fill(angles, mean_topics, alpha=0.25)

    ax.set_title(grp, pad=20)

    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(topic_cols, fontsize=9)

    ax.set_ylim(0, global_max * 1.1)   # ✅ shared scale
    ax.set_yticklabels([])

plt.suptitle("TN Subgroup Topic Weight Profiles (shared radial scale)", fontsize=14)
plt.tight_layout()
plt.show()


In [ ]:
## ADNI4 vulnerable check
print('VULN')
r2 = adni4_results[adni4_results['TN_group']=='Vulnerable']
print(r2['DX'].value_counts())
print('\nRESIL')
r2 = adni4_results[adni4_results['TN_group']=='Resilient']
print(r2['DX'].value_counts())
print('\nCANON')
r2 = adni4_results[adni4_results['TN_group']=='Canonical']
print(r2['DX'].value_counts())